Access MusAV/audio_chunks from my Google Drive's "shared with me" folder

In [ ]:
!pip install essentia-tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/291.4 MB 3.2 MB/s eta 0:01:17
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/urllib3/response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/cachecontrol/filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "/usr/lib/python3.8/http/client.py", line 459, in read
    n = self.readinto(b)
  File "/usr/lib/python3.8/http/client.py", line 503, in readinto
    n = self.fp.readinto(b)
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.8/ssl.py", line 1241, in recv_into
    return self.rea

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Don't run this cell if !pip install essentia-tensorflow works

# I use this when some Google servers error does not allow me to install
# Essentia on Google servers
# !wget -c https://files.pythonhosted.org/packages/28/87/46df62cc0d93b0cde76f400ec02ba210e23c177d76cdfac9a8b70c85e18c/essentia_tensorflow-2.1b6.dev858-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

# If Google servers for some reasons can not download and install essentia/essentia-tensorflow,
# I manually downloaded the wheels package and uploaded to my Google Drive, so use that
!pip install '/content/gdrive/MyDrive/Audio and Music processing lab - assignment/essentia_tensorflow-2.1b6.dev858-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./gdrive/MyDrive/Audio and Music processing lab - assignment/essentia_tensorflow-2.1b6.dev858-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
essentia-tensorflow is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [ ]:
import essentia.standard

import glob  # https://docs.python.org/3/library/glob.html   
import csv # https://docs.python.org/3/library/csv.html
import os
import statistics
from tqdm import tqdm
import json # get classification genre labels from DiscogsEffnet model .json file

Enter the global parent folder where the audio files to be analyzed are stored. The search for audio files is recursive. You can also change the audio file extension to be looked for.

In [ ]:
'''
In my case the global parent folder is;
/content/gdrive/MyDrive/Audio and Music processing lab - assignment/Task 2/MusAV/audio_chunks
'''
# you can change parent_folder_name as your global parent folder into which audio files will be searched recursively
parent_folder_name = '/content/gdrive/MyDrive/Audio and Music processing lab - assignment/Task 2/MusAV/'
audio_file_extension = '*.mp3'
full_parent_folder_glob_path = str(parent_folder_name + '**/' + audio_file_extension)

# use glob to recursively find audio files paths https://docs.python.org/3/library/glob.html
audio_files_pathnames = glob.glob(full_parent_folder_glob_path, recursive=True)
print(f'Found {len(audio_files_pathnames)} audio chunks files paths')
'''
unique_audio_files_pathnames = set(audio_files_pathnames)
print(f'Found {len(unique_audio_files_pathnames)} unique audio chunks files paths')
'''

Found 2100 audio chunks files paths


"\nunique_audio_files_pathnames = set(audio_files_pathnames)\nprint(f'Found {len(unique_audio_files_pathnames)} unique audio chunks files paths')\n"

In [ ]:
# enter path of Discogs-Effnet model file path
discogsEffnet_model_file_path = '/content/gdrive/MyDrive/Audio and Music processing lab - assignment/Task 2/ML Essentia models/Discogs-Effnet models/discogs-effnet-bs64-1.pb'
# enter path of Discogs-Effnet json file path for extracting genres labels names
discogsEffnet_json_file_path = '/content/gdrive/MyDrive/Audio and Music processing lab - assignment/Task 2/ML Essentia models/Discogs-Effnet models/discogs-effnet-bs64-1.json'
# enter path of Voice/instrumental model file path
voiceInstr_model_file_path = '/content/gdrive/MyDrive/Audio and Music processing lab - assignment/Task 2/ML Essentia models/Voice_Instrumental models/voice_instrumental-musicnn-msd-1.pb'
# enter path of Arousal/Valence model file path
vGG_embeddings_model_file_path = '/content/gdrive/MyDrive/Audio and Music processing lab - assignment/Task 2/ML Essentia models/Arousal_Valence models/audioset-vggish-3.pb'
arousalValence_model_file_path = '/content/gdrive/MyDrive/Audio and Music processing lab - assignment/Task 2/ML Essentia models/Arousal_Valence models/emomusic-vggish-audioset-1.pb'

In [ ]:
# look into the DiscogsEffnet .json file, retrieve the names of the genre classes
# and store them into a set
discogsEffnet_json_file = open(discogsEffnet_json_file_path)
discogsEffnet_json_file_data = json.load(discogsEffnet_json_file)
discogsEffnet_genre_classes = []
for i in discogsEffnet_json_file_data['classes']:
    discogsEffnet_genre_classes.append(i)
discogsEffnet_json_file.close()
print(f'Found {len(discogsEffnet_genre_classes)} genre classes in DiscogsEffent model .json file.')
print(discogsEffnet_genre_classes)

Found 400 genre classes in DiscogsEffent model .json file.
['Blues---Boogie Woogie', 'Blues---Chicago Blues', 'Blues---Country Blues', 'Blues---Delta Blues', 'Blues---Electric Blues', 'Blues---Harmonica Blues', 'Blues---Jump Blues', 'Blues---Louisiana Blues', 'Blues---Modern Electric Blues', 'Blues---Piano Blues', 'Blues---Rhythm & Blues', 'Blues---Texas Blues', 'Brass & Military---Brass Band', 'Brass & Military---Marches', 'Brass & Military---Military', "Children's---Educational", "Children's---Nursery Rhymes", "Children's---Story", 'Classical---Baroque', 'Classical---Choral', 'Classical---Classical', 'Classical---Contemporary', 'Classical---Impressionist', 'Classical---Medieval', 'Classical---Modern', 'Classical---Neo-Classical', 'Classical---Neo-Romantic', 'Classical---Opera', 'Classical---Post-Modern', 'Classical---Renaissance', 'Classical---Romantic', 'Electronic---Abstract', 'Electronic---Acid', 'Electronic---Acid House', 'Electronic---Acid Jazz', 'Electronic---Ambient', 'Electro

In [ ]:
# enter path of *.csv file where data will be dumped
# If file does not exist already, it will be created
csv_data_file_path = '/content/gdrive/MyDrive/Audio and Music processing lab - assignment/Task 2/MusAV - audio_chunks - data.csv'

# Since I am going to dump a dictionary into each row of the .csv file,
# here is basically where we determine the order of the column headers
fieldnames = ['Audio_file_path', 'Tempo_BPM']
fieldnames += discogsEffnet_genre_classes
fieldnames += ['Voice', 'Instrumental', 'Danceability', 'Arousal', 'Valence']

In [ ]:
did_csv_file_already_exist = os.path.isfile(csv_data_file_path)
# open the file in the append mode, so that it is not truncated every time this
# snippet is run. By doing so, if anything goes wrong during an execution of this
# code snippet, we can re-run it without starting over
csv_data_file_writer = open(csv_data_file_path, 'a')
csv_data_file_reader = open(csv_data_file_path, 'r')
csv_data_file_dictWriter = csv.DictWriter(csv_data_file_writer, fieldnames=fieldnames)
if did_csv_file_already_exist is False: # write header only if file was created right now and did not exist before
  print('No .csv file found at the given path. A .csv file has been created at the given path.')
  csv_data_file_dictWriter.writeheader()
else:
    print('.csv file found at the given path.')
# create a csv.DictReader and also create a set (containing unique values)
# data structure out of the values of the first field ('audio_file_path'), so that,
# when writing, we can check first whether each row is already present or not in the .csv file
csv_data_file_dictReader = csv.DictReader(csv_data_file_reader)
csv_data_file_already_existing_paths = set()
for row in csv_data_file_dictReader:
   this_row_dict = dict(row) # create a dictionary out of this row
   csv_data_file_already_existing_paths.add(this_row_dict['audio_file_path']) # add the value corresponding to the audio_file_path key to a set
print(f'{len(csv_data_file_already_existing_paths)} file paths already present in the .csv file. These files will not be analyzed ')
# print(csv_data_file_already_existing_paths)

No .csv file found at the given path. A .csv file has been created at the given path.
0 file paths already present in the .csv file. These files will not be analyzed 


ONLY FOR TEST PURPOSES, I DEFINE A BATCH SIZE IN ORDER TO RUN SCRIPTS FASTER WHEN I ONLY NEED TO TEST THE EXECUTION OF A FEW EXTRACTIONS

In [ ]:
# batch_size = len(audio_files_pathnames) - 1
batch_size = 10

# to make things easier for managing the Pandas' DataFrame later,
# I change the audio path to make it relative to the .py script for the Streamlit interface
# I will use the audio file paths as index of the DataFrame
def get_relative_path(audio_chunk_path): # takes the Google Drive path of a file and adapts it to relative path from this .py file
    return 'audio/' + audio_chunk_path[94:]

In [ ]:
# instantiate any extractor or TensorFlow model only once for all audio files,
# rather than once for each audio file
effnetDiscogs_model = essentia.standard.TensorflowPredictEffnetDiscogs(graphFilename=discogsEffnet_model_file_path)
voiceInstr_model = essentia.standard.TensorflowPredictMusiCNN(graphFilename=voiceInstr_model_file_path)
danceability_algo = essentia.standard.Danceability()

# Arousal/valence VGGish model
aV_inputLayerName = 'flatten_in_input'
aV_outputLayerName = 'dense_out'
embeddings_model = essentia.standard.TensorflowPredictVGGish(
    graphFilename = vGG_embeddings_model_file_path,
    input = 'model/Placeholder',
    output = 'model/vggish/embeddings')

av_model = essentia.standard.TensorflowPredict2D(
  graphFilename = arousalValence_model_file_path,
  input=  aV_inputLayerName,
  output = aV_outputLayerName)

In [ ]:
counter = 0
for audio_file_path in tqdm(audio_files_pathnames): # use tqdm progress bar functionality
  # add a row to the .csv file and start extracting features from an audio chunk
  # only if the .csv file does not already contain a row with the same file path
  if audio_file_path not in csv_data_file_already_existing_paths:
    audioSignal = essentia.standard.MonoLoader(filename=audio_file_path)()

    rhythmExt2013 = essentia.standard.RhythmExtractor2013(method="multifeature")
    bpm, beats, beats_confidence, _, beats_intervals = rhythmExt2013(audioSignal)

    effnetDiscogs_activations = effnetDiscogs_model(audioSignal)
    # print(f'Shape of effnetDiscogs_activations is {effnetDiscogs_activations.shape}')
    # effnetDiscogs_activations.shape is 82 * 400, 400 being the activations
    # so, for each column, we have to average over all 82 rows
    effnetDiscogs_activations_mean = []
    for activation_it in range(400):
      effnetDiscogs_activations_mean.append(statistics.mean(effnetDiscogs_activations[:,activation_it]))
    # print(f'The array containing the means of the Discogs-effnet activations is {len(effnetDiscogs_activations_mean)} items long')

    voiceInstr_activations = voiceInstr_model(audioSignal)
    # print(f'Shape of voiceInstr_activations is {voiceInstr_activations.shape}')
    # voiceInstr_activations.shape is 54 * 2, 2 being the activations
    voiceInstr_activations_mean = []
    for activation_it in range(2):
      voiceInstr_activations_mean.append(statistics.mean(voiceInstr_activations[:,activation_it]))
    # print(f'The array containing the means of the Voice/Instrumental activations is {len(voiceInstr_activations_mean)} items long')

    danceability, DFA = danceability_algo(audioSignal)

    # Arousal/valence model
    embeddings = embeddings_model(audioSignal)    
    av_activations = av_model(embeddings)
    # print(f'Arousal and valence activations have size; {av_activations.shape}')
    # av_activations.shape is 88 * 2, 88 being the time frames
    av_activations_mean = []
    for activation_it in range(2):
      av_activations_mean.append(statistics.mean(av_activations[:,activation_it]))

    file_features_dict = {             'Audio_file_path': get_relative_path(audio_file_path),
                                       'Tempo_BPM': bpm,
                                       'Voice': voiceInstr_activations_mean[0],
                                       'Instrumental': voiceInstr_activations_mean[1],
                                       'Danceability': danceability,
                                       'Arousal' : av_activations_mean[0],
                                       'Valence' : av_activations_mean[1]}
    file_features_dict.update(dict(zip(discogsEffnet_genre_classes, effnetDiscogs_activations_mean)))
    csv_data_file_dictWriter.writerow(file_features_dict)
    
    counter += 1

print(f'{counter} file analyzed out of {len(audio_files_pathnames)} files paths.')

# close the file
csv_data_file_reader.close()
csv_data_file_writer.close()

  0%|          | 1/2100 [00:05<3:01:53,  5.20s/it]

Arousal and valence activations have size; (88, 2)


  0%|          | 2/2100 [00:10<3:00:55,  5.17s/it]

Arousal and valence activations have size; (88, 2)


  0%|          | 3/2100 [00:15<3:04:04,  5.27s/it]

...

Arousal and valence activations have size; (88, 2)


  1%|          | 15/2100 [01:26<3:24:23,  5.88s/it]

Arousal and valence activations have size; (88, 2)


  1%|          | 15/2100 [01:28<3:23:55,  5.87s/it]


KeyboardInterrupt: ignored